# extract_clicks.ipynb

The beat times from the song samples are estimated from *librosa*, and appended as sparse binary arrays in [clicks](/clicks)

In [39]:
import sys
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt

In [40]:
def extract_clicks(path, genre=None, print_message=True, h_t=0.1):
    """
    Extract clicks from sound files in folder
    Genre shift is used to convert from indexing per genre (4 times 1-100) to indexing over all samples (1-400)
    There are some songs of shorter duration, their MFCC array will be filled with -1's 
    """

    sample_rate = 44100

    n_fft = 8192

    win_length = int(0.025*sample_rate)
    hop_length = int(0.010*sample_rate)

    duration = 60.00 # in seconds
    length_clicks = int(duration / h_t)
    
    number_of_samples_per_genre = 100

    CLICKS = np.full((number_of_samples_per_genre, length_clicks), np.nan)
    
    for file in os.scandir(path):
        if file.path.endswith(".mp3"):
            if print_message:
                print("analyzing ", file.name, file.path)

            currentpath = path + "/" + file.name
            int_track = int(file.name[:-4])

            sound, sample_rate = librosa.load(currentpath, sr=sample_rate, mono=True)
            duration_file = librosa.get_duration(sound)
            length_file = int(duration_file / (2*h_t)) 
            #For some reason, librosa is giving 2 times the duration of the file

            
            tempo, beat_times_librosa = librosa.beat.beat_track(sound,
                                                                sr=sample_rate, start_bpm=120, units='time')
            
            clicks = beat_times_librosa/h_t
            clicks = clicks.astype(np.int32)

            one_hot_clicks = np.zeros(length_file)

            one_hot_clicks[clicks] = 1
    
            if length_file < length_clicks:
                one_hot_clicks = np.append(one_hot_clicks, 
                                 np.full((length_clicks-length_file), np.nan),
                                axis=-1)
                
                print(one_hot_clicks)

            elif duration_file > duration:
                one_hot_clicks = one_hot_clicks[:length_clicks]

            CLICKS[int_track-1] = one_hot_clicks

    return CLICKS

In [41]:
path = "../data/emotifymusic/"
genres = ["classical", "electronic", "pop", "rock"]

CLICKS = []

for genre in genres:
    clicks = extract_clicks(path+genre, genre=genre)
    CLICKS.append(clicks)

analyzing  1.mp3 ../data/emotifymusic/classical\1.mp3
analyzing  10.mp3 ../data/emotifymusic/classical\10.mp3
analyzing  100.mp3 ../data/emotifymusic/classical\100.mp3
analyzing  11.mp3 ../data/emotifymusic/classical\11.mp3
analyzing  12.mp3 ../data/emotifymusic/classical\12.mp3
analyzing  13.mp3 ../data/emotifymusic/classical\13.mp3
analyzing  14.mp3 ../data/emotifymusic/classical\14.mp3
analyzing  15.mp3 ../data/emotifymusic/classical\15.mp3
analyzing  16.mp3 ../data/emotifymusic/classical\16.mp3
[ 1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.
  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.
  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.
  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.
  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  

In [42]:
print(np.shape(CLICKS))
CLICKS_reshaped = np.reshape(CLICKS, (np.shape(CLICKS)[0]*np.shape(CLICKS)[1], np.shape(CLICKS)[2]))

(4, 100, 600)


In [43]:
np.save("clicks/clicks", CLICKS_reshaped)